In [108]:
import yfinance as yf
import yoptions as yo
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

UNIVERSE = ["TSLA", "SPY", "XOM"]

def get_minute_data_for_28_days(ticker):
    days = 30
    ticker = yf.Ticker(ticker)
    hist = []
    for i in range(0, days):
        curr_day = (datetime.today() - timedelta(days=i)).strftime("%Y-%m-%d")
        prev_day = (datetime.today() - timedelta(days=i + 1)).strftime("%Y-%m-%d")
        hist.append(ticker.history(start=prev_day, end=curr_day, interval="1m"))
    return hist

def get_2minute_data_for_60_days(ticker):
    days = 60
    ticker = yf.Ticker(ticker)
    hist = []
    for i in range(30, days):
        curr_day = (datetime.today() - timedelta(days=i)).strftime("%Y-%m-%d")
        prev_day = (datetime.today() - timedelta(days=i + 1)).strftime("%Y-%m-%d")
        hist.append(ticker.history(start=prev_day, end=curr_day, interval="2m"))
    return hist
        
for ticker, contract_increment in zip(UNIVERSE, [2.5, 1, 1]):
    hist_1m = get_minute_data_for_28_days(ticker)
    hist_2m = get_2minute_data_for_60_days(ticker)
    hist_1m = pd.concat(hist_1m)
    hist_2m = pd.concat(hist_2m)
    total = pd.concat([hist_1m, hist_2m])
    total.drop_duplicates(inplace=True)
    total = total.reset_index().rename(columns={'index': 'Date'})
    total["Date"] = pd.to_datetime(total["Date"])
    total["Contract Strike"] = np.ceil(total["Close"] / contract_increment) * contract_increment
    total = total[["Date", "Contract Strike", "Close"]]
    total.sort_values(by=["Date"], inplace=True)
    total.to_csv(f"{ticker}.csv")


TSLA: No price data found, symbol may be delisted (1m 2023-12-03 -> 2023-12-04)
TSLA: No price data found, symbol may be delisted (1m 2023-12-02 -> 2023-12-03)
TSLA: No price data found, symbol may be delisted (1m 2023-11-26 -> 2023-11-27)
TSLA: No price data found, symbol may be delisted (1m 2023-11-25 -> 2023-11-26)
TSLA: No price data found, symbol may be delisted (1m 2023-11-23 -> 2023-11-24)
TSLA: No price data found, symbol may be delisted (1m 2023-11-19 -> 2023-11-20)
TSLA: No price data found, symbol may be delisted (1m 2023-11-18 -> 2023-11-19)
TSLA: No price data found, symbol may be delisted (1m 2023-11-12 -> 2023-11-13)
TSLA: No price data found, symbol may be delisted (1m 2023-11-11 -> 2023-11-12)
TSLA: 1m data not available for startTime=1699333200 and endTime=1699419600. The requested range must be within the last 30 days.
TSLA: No price data found, symbol may be delisted (2m 2023-11-05 -> 2023-11-06)
TSLA: No price data found, symbol may be delisted (2m 2023-11-04 -> 20

In [132]:
import scipy
from scipy.optimize import fmin
import time
import multiprocessing as mp

def black_scholes_call_price_vol(sigma, S, K, r, t, price):
    d1 = (np.log(S/K) + (r + sigma**2/2)*t)/(sigma*np.sqrt(t))
    d2 = d1 - sigma*np.sqrt(t)
    diff = scipy.stats.norm.cdf(d1)*S - scipy.stats.norm.cdf(d2)*K*np.exp(-r*t) - price
    diff = diff**2
    # print(f"[σ]={sigma}, Object Function Value: {diff}")
    return diff


expiries = [1/12, 2/12, 3/12, 4/12]
expiry_prices = {}
expiry_prices["TSLA"] = [9.13, 16.7, 20.4, 24.11]
expiry_prices["SPY"] = [3.72, 8.6, 11.48, 14.62]
expiry_prices["XOM"] = [1.72, 2.67, 3.8, 4.46]
def calculate_vol(ticker):
    df = pd.read_csv(f"{ticker}.csv")
    results_df = pd.DataFrame(columns=["Date", "Contract Strike", "Close", "exp", "price", "vol", "runtime"])
    for _, row in df.iterrows():
        S = row["Close"]
        K = row["Contract Strike"]
        r = 0.05
        for exp, price in zip(expiries, expiry_prices[ticker]):
            row[f"exp"] = exp
            row[f"price"] = price
            start = time.process_time_ns()
            sigma = fmin(black_scholes_call_price_vol, 0.3, args=(S, K, r, exp, price), disp=False)[0]
            end = time.process_time_ns()
            row["vol"] = sigma
            row["runtime"] = end - start
            results_df.loc[len(results_df.index)] = [row["Date"], row["Contract Strike"], row["Close"], exp, price, sigma, end - start]
    results_df.to_csv(f"{ticker}_vols.csv")
start = time.process_time_ns()
mp.Pool(3).map(calculate_vol, UNIVERSE)
end = time.process_time_ns()
print(f"Total Runtime: {(end - start) / 1e9} seconds")

Total Runtime: 286.75824 seconds


In [137]:
for ticker in UNIVERSE:
    df = pd.read_csv(f"{ticker}_vols.csv")
    # remove outliers
    std_dev = df["runtime"].std()
    mean = df["runtime"].mean()
    df = df[df["runtime"] < mean + 2 * std_dev]
    df = df[df["runtime"] > mean - 2 * std_dev]
    print(f"Average Runtime for {ticker}: {df['runtime'].mean() / 1e9} seconds")
    print(f"Standard Deviation for {ticker}: {df['runtime'].std() / 1e9} seconds")
    print(f"25% Quantile for {ticker}: {df['runtime'].quantile(0.25) / 1e9} seconds")
    print(f"75% Quantile for {ticker}: {df['runtime'].quantile(0.75) / 1e9} seconds")
    print()

Average Runtime for TSLA: 0.00147863002771231 seconds
Standard Deviation for TSLA: 0.0002163641323344417 seconds
25% Quantile for TSLA: 0.001338 seconds
75% Quantile for TSLA: 0.001617 seconds

Average Runtime for SPY: 0.001696614023023635 seconds
Standard Deviation for SPY: 0.00018789040585662431 seconds
25% Quantile for SPY: 0.001538 seconds
75% Quantile for SPY: 0.001819 seconds

Average Runtime for XOM: 0.0016057014875779574 seconds
Standard Deviation for XOM: 0.00018065378592964352 seconds
25% Quantile for XOM: 0.001449 seconds
75% Quantile for XOM: 0.001734 seconds



In [142]:
import matplotlib.pyplot as plt
tsla_df = pd.read_csv("TSLA_vols.csv")
spy_df = pd.read_csv("SPY_vols.csv")
xom_df = pd.read_csv("XOM_vols.csv")

# remove outliers
std_dev = tsla_df["runtime"].std()
mean = tsla_df["runtime"].mean()
tsla_df = tsla_df[tsla_df["runtime"] < mean + 2 * std_dev]
tsla_df = tsla_df[tsla_df["runtime"] > mean - 2 * std_dev]

std_dev = spy_df["runtime"].std()
mean = spy_df["runtime"].mean()
spy_df = spy_df[spy_df["runtime"] < mean + 2 * std_dev]
spy_df = spy_df[spy_df["runtime"] > mean - 2 * std_dev]

std_dev = xom_df["runtime"].std()
mean = xom_df["runtime"].mean()
xom_df = xom_df[xom_df["runtime"] < mean + 2 * std_dev]
xom_df = xom_df[xom_df["runtime"] > mean - 2 * std_dev]

plt.hist(tsla_df["runtime"] / 1e9, bins=25)
plt.xlabel("Runtime (seconds)")
plt.ylabel("Frequency")
plt.title("TSLA Runtime Distribution")
plt.savefig("tsla_runtime.png")
plt.clf()

plt.hist(spy_df["runtime"] / 1e9, bins=25)
plt.xlabel("Runtime (seconds)")
plt.ylabel("Frequency")
plt.title("SPY Runtime Distribution")
plt.savefig("spy_runtime.png")
plt.clf()

plt.hist(xom_df["runtime"] / 1e9, bins=25)
plt.xlabel("Runtime (seconds)")
plt.ylabel("Frequency")
plt.title("XOM Runtime Distribution")
plt.savefig("xom_runtime.png")
plt.clf()




<Figure size 640x480 with 0 Axes>